In [2]:
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random
from jax import device_put
%matplotlib inline

In [3]:
key = random.PRNGKey(0)

def selu(x, alpha=1.67, lmbda=1.05):
  return lmbda * jnp.where(x > 0, x, alpha * jnp.exp(x) - alpha)

x = random.normal(key, (1000000,))
%timeit selu(x).block_until_ready()

1.57 ms ± 523 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


key = random.PRNGKey(0): 这行代码创建了一个名为key的伪随机数生成器的实例，用于生成随机数。PRNGKey函数的参数0表示随机数种子。

def selu(x, alpha=1.67, lmbda=1.05): 这是一个自定义函数selu，它接受输入x以及两个可选参数alpha和lmbda。selu函数实现了SELU激活函数，用于对输入x进行处理。

x = random.normal(key, (1000000,)): 这一行使用key生成了一个包含1000000个随机数的数组x。这些随机数来自标准正态分布

这是一个Jupyter Notebook中的魔法命令，用于测量selu(x)函数的执行时间。它会多次运行函数以获得平均执行时间。block_until_ready()用于确保计算已经完成。这个命令将测量调用selu(x)的运行时间。

### 使用jit

In [4]:
selu_jit = jit(selu)
%timeit selu_jit(x).block_until_ready()

1.15 ms ± 198 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### 使用grad()

In [6]:
def sum_logistic(x):
  return jnp.sum(1.0 / (1.0 + jnp.exp(-x)))

x_small = jnp.arange(4.)
print(x_small)
derivative_fn = grad(sum_logistic) # 先得到导函数，而不是某点的导数
print(derivative_fn(x_small))

[0. 1. 2. 3.]
[0.25       0.19661194 0.10499357 0.04517666]


返回了在0 1 2 3四个点上的导数值大小

![函数图像](sum_logistic_plot.png)

grad() 和 jit() 可以任意混合使用。在上面的例子中，我们对 sum_logistic 进行了 jit，然后求出了它的导数。我们还可以更进一步：

In [7]:
print(grad(jit(grad(jit(grad(sum_logistic)))))(1.0))

-0.0353256


### 使用 vmap() 进行自动矢量化

vmap() 是 JAX 库中的一个函数，用于对函数进行矢量化映射，特别适用于将函数应用于数组的每个元素，以便高效处理批量数据。它可以用于自动并行化操作，从而提高计算效率。
基本语法：
import jax.numpy as jnp
from jax import vmap

定义一个函数

def my_function(x):
    return ...  # 某些操作，可能依赖于 x


使用 vmap() 对函数进行矢量化映射

vectorized_function = vmap(my_function)

对输入数组进行操作

output_array = vectorized_function(input_array)